# [Mini Project: Web Scraping]

## 주제: [미국 대도시의 인기순위별 식당 리스트 조사]

### 동기: 미국 대도시별 음식점 상권에 대해 알고싶어서

### 사용한 라이브러리
- webdriver
- sleep

In [3]:
from selenium import webdriver
from time import sleep
ch_driver = webdriver.Chrome('C:/Users/beave/AppData/Roaming/Microsoft/Windows/Start Menu/Programs/Python 3.7/chromedriver.exe')

ch_driver.implicitly_wait(5) ##이건 어디에 둬야하나. 한번 두면 계속 적용되는건가

yelp_url = 'https://www.yelp.com/'
ch_driver.get(yelp_url)

In [4]:
search_input = ch_driver.find_element_by_id('find_desc')
search_input.send_keys('restaurants')
city_input = ch_driver.find_element_by_id('dropperText_Mast')
city_input.click()
city_input.send_keys('los angeles')

search_button = ch_driver.find_element_by_id('header-search-submit')
search_button.click()

In [3]:
#일단 광고 게시물 제거해서 list로 만들기
def get_restaurant_list():
    result_list = []
   
   
    r_path = '//*[@id="wrap"]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div/ul/li/div/div/div/div/div[2]/div[1]/div[1]/div[1]'
    restaurants = ch_driver.find_elements_by_xpath(r_path)
    for i,item in enumerate(restaurants):
        title = item.find_element_by_tag_name('h3').text
        name = item.find_element_by_tag_name('a').text
        link = item.find_element_by_tag_name('a').get_attribute('href')

        #get rating
        rating_elem = ch_driver.find_element_by_xpath('//*[@id="wrap"]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div/ul/li[' +
                                                      str(i+1)+']/div/div/div/div/div[2]/div[1]/div[1]/div[1]/div[2]/span/div')
        rating = rating_elem.get_attribute('aria-label')
        #print(rating)

        #get number of reviews
        review_elem = ch_driver.find_element_by_xpath('//*[@id="wrap"]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div/ul/li['
                                                      + str(i+1)+']/div/div/div/div/div[2]/div[1]/div[1]/div[1]/div[3]/span')
        review_num = review_elem.text

        #get price index and category
        try:
            pc_elem = ch_driver.find_elements_by_xpath('//*[@id="wrap"]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div/ul/li[' +
                                                       str(i+1) + ']/div/div/div/div/div[2]/div[1]/div[1]/div[1]/div[4]/span')
            price = pc_elem[0].text
            category = pc_elem[1].text
        except IndexError:
            pc_elem = ch_driver.find_element_by_xpath('//*[@id="wrap"]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div/ul/li[' +
                                                       str(i+1) + ']/div/div/div/div/div[2]/div[1]/div[1]/div[1]/div[4]/span')
            price = "N/A"
            category = pc_elem.text
        
        #전체 var 받은뒤 title로 광고 게시물 거르기
        if title[0].isdigit() == False:
            continue
            
        result_list.append([name, link, rating, review_num, price, category])
        
    return result_list

In [4]:
#ranking이 key, list결과물을 value로 하는 dict 만들기 
def get_restaurants():
    result = {}
    result_list = get_restaurant_list()
    
    for i in range(30):
        result[str(i+1)] = (result_list[i])
        
    return result

In [5]:
all_result = {}
all_result['Los Angeles'] = get_restaurants()
sleep(5)

city_list = ['San Francisco', 'New York']

for city in city_list:
    #다른 도시로 이동
    another_city = ch_driver.find_element_by_id('search_location')
    another_city.click()
    another_city.send_keys(city)
    another_button = ch_driver.find_element_by_xpath('//*[@id="header_find_form"]/div/div[2]/div/div[2]/button')
    another_button.click()
    
    #그 도시 스크래핑한 내용 all_result에 담기
    all_result[city] = get_restaurants()
    sleep(5)

In [6]:
all_result

{'Los Angeles': {'1': ['Bestia',
   'https://www.yelp.com/biz/bestia-los-angeles?osq=restaurants',
   '4.5 star rating',
   '5343 reviews',
   '$$$',
   'Italian, Cocktail Bars, Pizza'],
  '2': ['Howlin’ Ray’s',
   'https://www.yelp.com/biz/howlin-rays-los-angeles-3?osq=restaurants',
   '4.5 star rating',
   '4313 reviews',
   '$$',
   'Southern, Chicken Shop, American (Traditional)'],
  '3': ['Running Goose',
   'https://www.yelp.com/biz/running-goose-los-angeles?osq=restaurants',
   '4.5 star rating',
   '757 reviews',
   '$$',
   'Tapas/Small Plates, Gastropubs, Latin American'],
  '4': ['République',
   'https://www.yelp.com/biz/r%C3%A9publique-los-angeles-2?osq=restaurants',
   '4 star rating',
   '4453 reviews',
   '$$',
   'French, Breakfast & Brunch, Cocktail Bars'],
  '5': ['RockSugar Southeast Asian Kitchen',
   'https://www.yelp.com/biz/rocksugar-southeast-asian-kitchen-los-angeles?osq=restaurants',
   '4 star rating',
   '2177 reviews',
   '$$',
   'Indonesian, Vietnamese, 

In [8]:
all_result['San Francisco']['10'] #San Francisco 10위 식당 정보

['Hog Island Oyster Co',
 'https://www.yelp.com/biz/hog-island-oyster-co-san-francisco?osq=restaurants',
 '4.5 star rating',
 '5454 reviews',
 '$$',
 'Seafood, Seafood Markets, Live/Raw Food']

## 느낀 점

### 어려웠던 점
1. 처음엔 API로 하고싶었는데 해당 파일?이 뭔지 찾기 힘들었음
2. 그래서 Selenium으로 하려는데 각 사이트마다 tag형식?구조?가 다 달라서 크롤링할 사이트를 정하는게 시간이 오래 걸렸음. (도시별로 검색하기에 적합한지, 긁어오기에 적합한지 등을 판단해야해서)
3. 시행착오 끝에 Yelp로 정했는데 광고를 걸러내야해서 번거로웠음. 처음엔 광고가 항상 앞에 세개씩 있는건가 해서 앞에 세개만 짤랐었는데 도시별로 광고 수가 달라서 그냥 전부 다 긁어온 다음에 광고가 있는걸 빼는 방법으로 함
4. 이런 등등의 사이트별 규칙?을 파악하는게 가장 힘들었던것 같음(Xpath 같은것)

### 보완할 부분
긁어올 부분의 규칙이나 구조 등을 빨리 파악하도록 해야할것같다. 
### 소감
그냥 검색을 하는것도 나쁘지 않다고 생각합니다....익숙한 사이트가 아니라면...